In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_excel('tweet_banjir.xlsx', header=0)
df.head()

,id_str,from_user,text,created_at,time,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,user_followers_count,user_friends_count,user_location
0,1357116635094474754,pelindo3,Pelindo III Peduli Banjir Sekotong\n-\nHalo #P...,Wed Feb 03 23:59:54 +0000 2021,2021-02-03 23:59:54,NaN,NaN,81799468,NaN,11582.0,344.0,Indonesia
1,1357116624285749253,hye_bye_99,RT @luqmanlong: Bab orang sentuh pangkat Datuk...,Wed Feb 03 23:59:51 +0000 2021,2021-02-03 23:59:51,NaN,NaN,1741183075,NaN,90.0,184.0,"Johor Bahru, Johor"
2,1357116610280968194,anwar_muhmmd,"@pln_123 Sudah min, banjir juga ws surut",Wed Feb 03 23:59:48 +0000 2021,2021-02-03 23:59:48,5.585161e+08,pln_123,1088076279331512320,1.357097e+18,1.0,52.0,"Kota Probolinggo, Jawa Timur"
3,1357116566752530432,petabencana,"@BangJac63679229 Halo, saya Bencana Bot. Untuk...",Wed Feb 03 23:59:38 +0000 2021,2021-02-03 23:59:38,1.345356e+18,BangJac63679229,2276880895,1.357117e+18,122065.0,311.0,Indonesia
4,1357116555188871168,milokasaka,RT @hxppypom_: Tweet macam ni selalu tak viral...,Wed Feb 03 23:59:35 +0000 2021,2021-02-03 23:59:35,NaN,NaN,1036594916,NaN,277.0,297.0,Mirul's


In [4]:
def clean_lower(lwr):
    lwr = lwr.lower()
    return lwr

df['lwr'] = df['text'].apply(clean_lower)
casefolding = pd.DataFrame(df['lwr'])
casefolding

,lwr
0,pelindo iii peduli banjir sekotong\n-\nhalo #p...
1,rt @luqmanlong: bab orang sentuh pangkat datuk...
2,"@pln_123 sudah min, banjir juga ws surut"
3,"@bangjac63679229 halo, saya bencana bot. untuk..."
4,rt @hxppypom_: tweet macam ni selalu tak viral...
...,...
2933,"@alfahrifamily halo, saya bencana bot. untuk m..."
2934,rt @itsmirah__: @cacatcintayet @k31019_ kita s...
2935,"rt @kartelego: ayo habiskan hutanmu, entah pak..."
2936,"sedih deh ketika banjir gini, seketika mikir k..."


In [5]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text# Buat kolom tambahan untuk data description yang telah diremovepunctuation   
df['clean_punct'] = df['lwr'].apply(clean_punct)
df['clean_punct']

0       pelindo iii peduli banjir sekotong   halo  por...
1       rt  luqmanlong  bab orang sentuh pangkat datuk...
2                  pln 123 sudah min banjir juga ws surut
3        bangjac63679229 halo saya bencana bot  untuk ...
4       rt  hxppypom   tweet macam ni selalu tak viral...
                              ...                        
2933     alfahrifamily halo saya bencana bot  untuk me...
2934    rt  itsmirah     cacatcintayet  k31019  kita s...
2935    rt  kartelego  ayo habiskan hutanmu entah paka...
2936    sedih deh ketika banjir gini seketika mikir ke...
2937                                        banjir astaga
Name: clean_punct, Length: 2938, dtype: object

In [6]:
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
df['clean_double_ws'] = df['clean_punct'].apply(_normalize_whitespace)
df['clean_double_ws']

0       pelindo iii peduli banjir sekotong halo portiz...
1       rt luqmanlong bab orang sentuh pangkat datuk t...
2                  pln 123 sudah min banjir juga ws surut
3       bangjac63679229 halo saya bencana bot untuk me...
4       rt hxppypom tweet macam ni selalu tak viral ba...
                              ...                        
2933    alfahrifamily halo saya bencana bot untuk mela...
2934    rt itsmirah cacatcintayet k31019 kita still ad...
2935    rt kartelego ayo habiskan hutanmu entah pakai ...
2936    sedih deh ketika banjir gini seketika mikir ke...
2937                                        banjir astaga
Name: clean_double_ws, Length: 2938, dtype: object

In [7]:
#clean stopwords
stopword = set(stopwords.words('indonesian'))
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text# Buat kolom tambahan untuk data description yang telah distopwordsremoval   
df['clean_sw'] = df['clean_double_ws'].apply(clean_stopwords)
df['clean_sw']

0       pelindo iii peduli banjir sekotong halo portiz...
1       rt luqmanlong bab orang sentuh pangkat datuk t...
2                             pln 123 min banjir ws surut
3       bangjac63679229 halo bencana bot melaporkan ba...
4       rt hxppypom tweet ni viral dah menteri agama t...
                              ...                        
2933    alfahrifamily halo bencana bot melaporkan banj...
2934    rt itsmirah cacatcintayet k31019 still ydpa be...
2935    rt kartelego ayo habiskan hutanmu pakai alasan...
2936    sedih deh banjir gini mikir keras berangkat ke...
2937                                        banjir astaga
Name: clean_sw, Length: 2938, dtype: object

In [8]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['sudah_bersih'] = df['clean_sw'].apply(stemmer.stem)
df['sudah_bersih']

KeyboardInterrupt: 

In [3]:
lemmatization=pd.DataFrame(df['sudah_bersih'])
token=nltk.tokenize.WhitespaceTokenizer().tokenize(lemmatization['text'][100])
token

['Topik', 'di', 'wag', 'keluarga', 'masih', 'banjir', 'aja']